<a href="https://colab.research.google.com/github/markNZed/GPT-NeoX-Colab/blob/venv/notebooks/shakespeare_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Training a tiny LLM on a corpus of Shakespeare

In [ ]:
from datetime import datetime
print("Current Date and Time:", datetime.now())

In [ ]:
# We could modify these paths to "stub" behavior for test/dev
workspaceDir = "/content"
GPTNeoXColabDirName = "GPT-NeoX-Colab"
GPTNeoXColabDir = f"{workspaceDir}/{GPTNeoXColabDirName}"

# Cloning Git Repos

In [ ]:
#@title Clone GPT-NeoX-Colab
%%time
%cd {workspaceDir}
# Don't use --depth 1 because that does not play nice with git-annex
!git clone -b venv https://github.com/markNZed/GPT-NeoX-Colab.git
%cd {GPTNeoXColabDir}
!pip install . > /dev/null 2>&1
from dotenv import load_dotenv
import os
load_dotenv(f"{GPTNeoXColabDir}/.env")
import GPTNeoXColab
GPTNeoXColab.utils.colab.install_git_annex()
GPTNeoXColab.utils.colab.enable_remote()
GPTNeoXColab.utils.colab.sync_annex()
GPTNeoXColab.utils.colab.fetch_data("data/shakespeare/shakespeare.txt")
GPTNeoXColab.utils.colab.fetch_data("data/shakespeare/shakespeare.jsonl")
GPTNeoXColab.utils.colab.fetch_data("data/shakespeare/shakespeare_text_document.bin")
GPTNeoXColab.utils.colab.fetch_data("data/shakespeare/shakespeare_text_document.idx")

# Inference with Hugging Face

In [ ]:
from transformers import GPTNeoXForCausalLM
import torch

# Move to model directory
%cd {GPTNeoXDir}

# Assuming CharLevelTokenizer is properly imported and instantiated
from GPTNeoXColab import CharLevelTokenizer
tokenizer = CharLevelTokenizer.CharLevelTokenizer(vocab_size=512)

# Load your model
model_path = f"{GPTNeoXColabDir}/data/shakespeare"
model = GPTNeoXForCausalLM.from_pretrained(model_path)

# Define a simple char-level tokenizer if not provided
def char_level_tokenize(text):
    return tokenizer.tokenize(text)

def char_level_detokenize(tokens):
    return tokenizer.detokenize(tokens)

# Set the model to evaluation mode
model.eval()

# Prompt the user for input
input_text = input("Enter your prompt: ")

# Tokenize and prepare input
input_ids = torch.tensor([char_level_tokenize(input_text)], dtype=torch.long)
attention_mask = torch.ones_like(input_ids)  # Create an attention mask for non-padded input

# Generate text with specified pad_token_id and attention_mask
with torch.no_grad():
    output = model.generate(
        input_ids,
        attention_mask=attention_mask,
        max_length=200,          # Adjust this for desired output length
        temperature=0.7,        # Controls creativity
        top_k=50,               # Controls diversity
        top_p=0.9,              # Nucleus sampling
        num_return_sequences=1, # Number of sequences to return
        pad_token_id=model.config.eos_token_id,  # Set pad_token_id explicitly
        do_sample=True           # Enable sampling mode to use temperature and top_p
    )

# Decode and print the generated text
generated_text = char_level_detokenize(output[0].tolist())
print("Generated text:", generated_text)